El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

Antes de poder trabajar en crear nuestro modelo, debemos de revisar si toda la información provista es pertinente.
También es necesario darle estructura en caso de que los títulos de las columnas nos resulten confusos.
Haremos una limpieza de los datos provistos y nos quedaremos únicamente con la información útil para entrenar nuestro modelo.

In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd #Instalamos liberías necesarias para entrenar nuestro modelo
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [4]:
df = pd.read_csv('/datasets/car_data.csv') #Leemos data
df = df.rename(columns={'DateCrawled':'download_profile_date','Price':'price','VehicleType':'vehicle_type','RegistrationYear':'reg_year','Gearbox':'gearbox','Power':'power_cv','Model':'vehicle_model','Mileage':'mileage','RegistrationMonth':'vehicle_reg_month','FuelType':'fuel_type','Brand':'brand','NotRepaired':'not_repaired','DateCreated':'profile_date_creation','NumberOfPictures':'number_of_pics','PostalCode':'postal_code','LastSeen':'user_last_active_date'}) #Renombramos columnas para facilitar lectura
print(df.info()) #Revisamos contenidos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   download_profile_date  354369 non-null  object
 1   price                  354369 non-null  int64 
 2   vehicle_type           316879 non-null  object
 3   reg_year               354369 non-null  int64 
 4   gearbox                334536 non-null  object
 5   power_cv               354369 non-null  int64 
 6   vehicle_model          334664 non-null  object
 7   mileage                354369 non-null  int64 
 8   vehicle_reg_month      354369 non-null  int64 
 9   fuel_type              321474 non-null  object
 10  brand                  354369 non-null  object
 11  not_repaired           283215 non-null  object
 12  profile_date_creation  354369 non-null  object
 13  number_of_pics         354369 non-null  int64 
 14  postal_code            354369 non-null  int64 
 15  

Decidí eliminar estas columnas ya que no creo que sean relevantes para la predicción del precio del automovil.
Las variables dentro de estas columnas no nos sirvan para hacer un modelo como el que buscamos.

In [5]:
df = df.drop(['download_profile_date', 'profile_date_creation', 'number_of_pics', 'postal_code', 'user_last_active_date', 'vehicle_reg_month'], axis=1)

In [6]:
print(df.isna().sum()) #Revisamos datos nulos en nuestro DataFrame
print()
print(df.duplicated().sum()) #Revisamos si existen datos duplicados

price                0
vehicle_type     37490
reg_year             0
gearbox          19833
power_cv             0
vehicle_model    19705
mileage              0
fuel_type        32895
brand                0
not_repaired     71154
dtype: int64

45040


Llenamos los valores núlos con un valor de Unkown, ya que no sabemos que información corresponde en estas columnas.
Información como tipo de vehículo, caja de cambios, modelo y tipo de combustible, son variables no numéricas que el usuarios nos debería de proveer.

In [7]:
df = df.fillna('Unknown') 

In [8]:
df = df.drop_duplicates() #Removemos datos duplicados de nuestra base de datos
print(df.duplicated().sum()) #Validamos que ya no existan duplicados
print()
print(df.head(5)) #Checamos como quedó nuestro data frame

0

   price vehicle_type  reg_year gearbox  power_cv vehicle_model  mileage  \
0    480      Unknown      1993  manual         0          golf   150000   
1  18300        coupe      2011  manual       190       Unknown   125000   
2   9800          suv      2004    auto       163         grand   125000   
3   1500        small      2001  manual        75          golf   150000   
4   3600        small      2008  manual        69         fabia    90000   

  fuel_type       brand not_repaired  
0    petrol  volkswagen      Unknown  
1  gasoline        audi          yes  
2  gasoline        jeep      Unknown  
3    petrol  volkswagen           no  
4  gasoline       skoda           no  


Creamos otro data frame que contenga sólo lo9s datos que sí conocemos. De esa manera podemos entrenar el modelo con variables conocidas y comparar los resultados.

In [9]:
df_no_unknown= df[(df.vehicle_type != 'Unknown') & (df.gearbox != 'Unknown') & (df.vehicle_model!= 'Unknown') & (df.fuel_type != 'Unknown') & (df.not_repaired != 'Unknown')]
print(df_no_unknown.head(5))#Validamos que los datos unkown hayan sido removidos.

   price vehicle_type  reg_year gearbox  power_cv vehicle_model  mileage  \
3   1500        small      2001  manual        75          golf   150000   
4   3600        small      2008  manual        69         fabia    90000   
5    650        sedan      1995  manual       102           3er   150000   
6   2200  convertible      2004  manual       109       2_reihe   150000   
7      0        sedan      1980  manual        50         other    40000   

  fuel_type       brand not_repaired  
3    petrol  volkswagen           no  
4  gasoline       skoda           no  
5    petrol         bmw          yes  
6    petrol     peugeot           no  
7    petrol  volkswagen           no  


In [10]:
df.describe()

,price,reg_year,power_cv,mileage
count,309329.000000,309329.000000,309329.000000,309329.000000
mean,4486.937196,2004.360105,110.976908,127217.735809
std,4564.852796,92.541399,200.969473,38532.941010
min,0.000000,1000.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,69.000000,125000.000000
50%,2800.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


In [11]:
df_no_unknown.describe()

,price,reg_year,power_cv,mileage
count,209576.000000,209576.000000,209576.000000,209576.000000
mean,5261.494527,2002.991278,121.071091,126003.192159
std,4779.064833,6.305215,149.265029,37817.461136
min,0.000000,1910.000000,0.000000,5000.000000
25%,1500.000000,1999.000000,76.000000,100000.000000
50%,3600.000000,2004.000000,114.000000,150000.000000
75%,7750.000000,2007.000000,150.000000,150000.000000
max,20000.000000,2018.000000,20000.000000,150000.000000


Revisamos la descripción de nuestros dataframes para conocer los distintos percentiles, mínimos y máximos.
Al parecer nuestro dataframe "df" contiene información que no hace sentido.
Por ejemplo, tenemos años de registro en el año 1,000 y 9,999 lo cual son rangos muy altos que no son necsearios.
También existen autos sin Power CV, esto no debería ser posible ya que todos los autos tienen caballos de fuerza.
Según la página oficial de Mapfre el mínimo de CV permitido para cualquier tipo de auto es de 60CV y el rango máximo es de 2332CV según Gemini de Google, usaremos estos rangos para filtrar nuestro dataframe.

In [12]:
df = df[(df['reg_year'] >= 1910) & (df['reg_year'] <= 2024)]
df = df[(df['power_cv'] >= 60) & (df['power_cv'] <= 2332)]
df.describe()

,price,reg_year,power_cv,mileage
count,253600.000000,253600.000000,253600.000000,253600.000000
mean,5034.294231,2003.643178,127.715398,127821.451104
std,4690.193804,6.847204,66.423579,37335.928628
min,0.000000,1910.000000,60.000000,5000.000000
25%,1450.000000,1999.000000,90.000000,125000.000000
50%,3450.000000,2004.000000,116.000000,150000.000000
75%,7300.000000,2008.000000,150.000000,150000.000000
max,20000.000000,2019.000000,2331.000000,150000.000000


In [13]:
df_no_unknown = df_no_unknown[(df_no_unknown['power_cv'] >= 60) & (df_no_unknown['power_cv'] <= 2332)]
df_no_unknown.describe()

,price,reg_year,power_cv,mileage
count,188914.000000,188914.000000,188914.000000,188914.000000
mean,5601.949898,2003.428168,129.470870,125985.845411
std,4826.988275,5.992009,61.549348,37810.700567
min,0.000000,1930.000000,60.000000,5000.000000
25%,1799.000000,2000.000000,90.000000,100000.000000
50%,3999.000000,2004.000000,117.000000,150000.000000
75%,8150.000000,2008.000000,150.000000,150000.000000
max,20000.000000,2018.000000,2331.000000,150000.000000


Como recomendación de mi tutor voy a transformar las columnas de 'brand' y 'vehicle_model' en variables ordinarias.
Así hacemos más pequeño el dataframe de con las características dummy para entranar el modelo.

In [14]:
df[['brand', 'vehicle_model']] = OrdinalEncoder().fit_transform(df[['brand', 'vehicle_model']])
df_no_unknown[['brand', 'vehicle_model']] = OrdinalEncoder().fit_transform(df_no_unknown[['brand', 'vehicle_model']])

In [15]:
df.head()

,price,vehicle_type,reg_year,gearbox,power_cv,vehicle_model,mileage,fuel_type,brand,not_repaired
1,18300,coupe,2011,manual,190,25.0,125000,gasoline,1.0,yes
2,9800,suv,2004,auto,163,117.0,125000,gasoline,14.0,Unknown
3,1500,small,2001,manual,75,116.0,150000,petrol,38.0,no
4,3600,small,2008,manual,69,101.0,90000,gasoline,31.0,no
5,650,sedan,1995,manual,102,11.0,150000,petrol,2.0,yes


In [16]:
df_no_unknown.head()

,price,vehicle_type,reg_year,gearbox,power_cv,vehicle_model,mileage,fuel_type,brand,not_repaired
3,1500,small,2001,manual,75,115.0,150000,petrol,36.0,no
4,3600,small,2008,manual,69,100.0,90000,gasoline,31.0,no
5,650,sedan,1995,manual,102,11.0,150000,petrol,2.0,yes
6,2200,convertible,2004,manual,109,8.0,150000,petrol,25.0,no
10,2000,sedan,2004,manual,105,10.0,150000,petrol,19.0,no


In [17]:
variables_categoricas = ['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired']

In [18]:
df_model = pd.get_dummies(df, drop_first=True, columns=variables_categoricas) #Volvemos las variables categóricas en booleanos

print(df_model.head(5)) #Revisamos como queda nuestro dataframe

   price  reg_year  power_cv  vehicle_model  mileage  brand  vehicle_type_bus  \
1  18300      2011       190           25.0   125000    1.0                 0   
2   9800      2004       163          117.0   125000   14.0                 0   
3   1500      2001        75          116.0   150000   38.0                 0   
4   3600      2008        69          101.0    90000   31.0                 0   
5    650      1995       102           11.0   150000    2.0                 0   

   vehicle_type_convertible  vehicle_type_coupe  vehicle_type_other  ...  \
1                         0                   1                   0  ...   
2                         0                   0                   0  ...   
3                         0                   0                   0  ...   
4                         0                   0                   0  ...   
5                         0                   0                   0  ...   

   gearbox_manual  fuel_type_cng  fuel_type_electric  fu

In [19]:
df_model_no_unknown = pd.get_dummies(df_no_unknown, drop_first=True, columns=variables_categoricas) #Hacemos booleanos para dataframe sin unknowns

print(df_model_no_unknown.head(5)) #Revisamos como queda nuestro dataframe

    price  reg_year  power_cv  vehicle_model  mileage  brand  \
3    1500      2001        75          115.0   150000   36.0   
4    3600      2008        69          100.0    90000   31.0   
5     650      1995       102           11.0   150000    2.0   
6    2200      2004       109            8.0   150000   25.0   
10   2000      2004       105           10.0   150000   19.0   

    vehicle_type_convertible  vehicle_type_coupe  vehicle_type_other  \
3                          0                   0                   0   
4                          0                   0                   0   
5                          0                   0                   0   
6                          1                   0                   0   
10                         0                   0                   0   

    vehicle_type_sedan  ...  vehicle_type_suv  vehicle_type_wagon  \
3                    0  ...                 0                   0   
4                    0  ...                 

## Entrenamiento del modelo 

Ya que tenemos la data limpia y necesaria para crear nuestro modelo, podemo empezar a desarrollarlo y entrenarlo con la data.
Crearemos distintos tipos de modelos y evaluaremos cual sería el mejor para este problema en específico.
También entrenaremos el modelo con la información disponible y también lo entrenaremos quitando los valores de Unknown.
Ya que estos datos faltantes podrían afectar la eficiencia del modelo al hacer predicciones, evaluaremos cual es la mejor opción.

Los dataframes con dummies nos servirán para entrenar una regresión lineal y un bosque de decisiones.
Los modelos de CatBoost y LighTGBM no necesitan este tratamiento así que podemos entrenarlos con datos que no fueron tratados como booleanos.

Por sugerencia de mi mentos volví a separar los datos sets para tener un data set de prueba adicional.
También se simplificaron las variables y el contenido dentro de estas.

Por motivos prácticos usaré el dataframe que contiene la información de Unknown, para entrenar los modelos cuando no tenemos toda la información disponible dentro de las variables.

In [20]:
train_valid, test = train_test_split(df_model, test_size=0.30, random_state=12345)
train, valid = train_test_split(train_valid, test_size=0.30, random_state=54321)
#Dividimos data sets de entrenamiento, validación y prueba.
#Estos datasets están codificados con el método One Hot

In [21]:
feat_train = train.drop('price', axis=1)
target_train = train['price']

feat_valid = valid.drop('price', axis=1)
target_valid = valid['price']

feat_test = test.drop('price', axis=1)
target_test = test['price']

In [22]:
df[variables_categoricas] = df[variables_categoricas].astype('category')
#Para poder usar los modelos de CatBoost y LightGBM las variables categoricas deben ser cambiadas a tipo de datos de categoría

In [23]:
train_valid_cat, test_cat = train_test_split(df, test_size=0.30, random_state=12345)
train_cat, valid_cat = train_test_split(train_valid_cat, test_size=0.30, random_state=54321)
#Dividimos data sets de entrenamiento, validación y prueba.
#Estos datasets no están codificados con dummies One Hot, se usarán para probar los modelos CatBoost y LightGBM

In [24]:
feat_train_cat = train_cat.drop('price', axis=1)
target_train_cat = train_cat['price']

feat_valid_cat = valid_cat.drop('price', axis=1)
target_valid_cat = valid_cat['price']

feat_test_cat = test_cat.drop('price', axis=1)
target_test_cat = test_cat['price']

In [25]:
def rmse(valid, predict): #Definimos función que nos ayude a encontrar el error cuadrático medio
    return math.sqrt(mean_squared_error(valid, predict))

## Análisis del modelo

### Modelo de regresión LightGBM

Para entrenar el modelo de LightGBM decidí usar los siguientes hiperparámetro:

num_iterations = el cual nos dicta el número de iteraciones que hace el modelo para reducir la métrica de error.

num_leaves = es la cantidad de nodos que tendrá cada árbol, mientras que la profundidad nos da la cantidad de niveles. Los nodos son las decisiones las que llega la regresión sin tener más ramificaciones.

max_depth = es la profundidad máxima por árbol de regresión dentro del algoritmo.

verbose = usamos verbose para que el algoritmo nos arroje información detalladas sobre como está procesando la información nuestro algoritmo.

metric = aqui indicamos cual va a ser nuestra métrica de error, varias de estas métricas ya están incluidas dentro de este hiperparámetro. Incluyendo RMSE, el error que queremos minimizar en este proyecto.

random_state = establecemos que la data entre de manera aleatoria al momento de crear cada árbol de regresión.

In [26]:
light_model = LGBMRegressor(num_iterations=20, num_leaves=25, max_depth=5, verbose=1, metric='rmse', random_state=12345)

In [27]:
light_model.fit(feat_train_cat, target_train_cat, eval_set=(feat_valid_cat, target_valid_cat), categorical_feature=variables_categoricas)

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['fuel_type', 'gearbox', 'not_repaired', 'vehicle_type']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 645
[LightGBM] [Info] Number of data points in the train set: 124264, number of used features: 9
[LightGBM] [Info] Start training from score 5030.055970
[1]	valid_0's rmse: 4382.71
[2]	valid_0's rmse: 4109.16
[3]	valid_0's rmse: 3868.66
[4]	valid_0's rmse: 3662.33
[5]	valid_0's rmse: 3477.22
[6]	valid_0's rmse: 3309.59
[7]	valid_0's rmse: 3168.01
[8]	valid_0's rmse: 3042.64


/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[9]	valid_0's rmse: 2936.61
[10]	valid_0's rmse: 2840.67
[11]	valid_0's rmse: 2755.96
[12]	valid_0's rmse: 2683.1
[13]	valid_0's rmse: 2619.23
[14]	valid_0's rmse: 2559.29
[15]	valid_0's rmse: 2509.05
[16]	valid_0's rmse: 2462.09
[17]	valid_0's rmse: 2423.11
[18]	valid_0's rmse: 2384.74
[19]	valid_0's rmse: 2354.12
[20]	valid_0's rmse: 2322.63


LGBMRegressor(max_depth=5, metric='rmse', num_iterations=20, num_leaves=25,
              random_state=12345, verbose=1)

In [28]:
print("Score LightGBM set de Entrenamiento: ", light_model.score(feat_train_cat, target_train_cat))
print("Score LightGBM set de Validación: ", light_model.score(feat_valid_cat, target_valid_cat))
print("Score LightGBM set de Prueba: ", light_model.score(feat_test_cat, target_test_cat))

Score LightGBM set de Entrenamiento:  0.759513420181545
Score LightGBM set de Validación:  0.7553591686321017
Score LightGBM set de Prueba:  0.7563352879734395


In [29]:
start_time_light = time.time()
light_model_predict = light_model.predict(feat_test_cat)
predict_time_light = time.time() - start_time_light
print(f"Predicting time light model: {predict_time_light:.4f} seconds")
print()
print(f"RMSE test light model: {rmse(target_test_cat, light_model_predict)}")

Predicting time light model: 0.0990 seconds

RMSE test light model: 2317.5618088798815


#### Conclusiones modelo LightGBM

Este modelo de algoritmo ya contiene el cálculo de gradiente y un parámetro para evaluar alguna métrica de error.
También cuanta con la bondad de no tener codificar al estilo One Hot antes de utilizar este algoritmo.
Otro beneficio de este modelo esque tiene un tiempo de entrenamiento corto.
A pesar de eso tiene dificultad en leer la data y las evaluaciones con los datos de validación y prueba apenas llegan a un nivel satisfactorio.

### Modelo de regresión CatBoost

Para entrenar el modelo de Catboost decií usar los siguientes hiperparámetro:

iterations = el cual nos dicta el número de iteraciones que hace el modelo para reducir la métrica de error.

max_depth = es la profundidad máxima por árbol de regresión dentro del algoritmo.

learning_rate = afecta el tiempo que tarda el algoritmo en entrenarse, mientras mayor sea la iteración menor es el learning rate.

loss_function = aqui indicamos cual va a ser nuestra métrica de error, varias de estas métricas ya están incluidas dentro de este hiperparámetro. Incluyendo RMSE, el error que queremos minimizar en este poryecto.

random_state = establecemos que la data entre de manera aleatoria al momento de crear cada árbol de regresión.

In [41]:
cat_model = CatBoostRegressor(iterations=20, max_depth=5, learning_rate=0.6, loss_function='RMSE', random_state=12345, cat_features=variables_categoricas)

In [42]:
cat_model.fit(feat_train_cat, target_train_cat)

0:	learn: 3211.4993388	total: 38.5ms	remaining: 731ms
1:	learn: 2692.2990815	total: 77.2ms	remaining: 695ms
2:	learn: 2518.9071653	total: 112ms	remaining: 633ms
3:	learn: 2409.0098537	total: 145ms	remaining: 579ms
4:	learn: 2343.8156397	total: 177ms	remaining: 530ms
5:	learn: 2293.1285302	total: 207ms	remaining: 483ms
6:	learn: 2231.9395822	total: 238ms	remaining: 441ms
7:	learn: 2208.5658698	total: 269ms	remaining: 404ms
8:	learn: 2177.3337590	total: 298ms	remaining: 365ms
9:	learn: 2144.3610804	total: 327ms	remaining: 327ms
10:	learn: 2126.4297501	total: 358ms	remaining: 293ms
11:	learn: 2111.6625073	total: 389ms	remaining: 259ms
12:	learn: 2095.2591197	total: 418ms	remaining: 225ms
13:	learn: 2070.7257832	total: 451ms	remaining: 193ms
14:	learn: 2062.8392709	total: 481ms	remaining: 160ms
15:	learn: 2053.7226697	total: 510ms	remaining: 128ms
16:	learn: 2045.6973202	total: 544ms	remaining: 96ms
17:	learn: 2032.1541898	total: 577ms	remaining: 64.1ms
18:	learn: 2020.6107021	total: 609ms

In [43]:
print("Score Catboost set de Entrenamiento: ", cat_model.score(feat_train_cat, target_train_cat))
print("Score Catboost set de Validación: ", cat_model.score(feat_valid_cat, target_valid_cat))
print("Score Catboost set de Prueba: ", cat_model.score(feat_test_cat, target_test_cat))

Score Catboost set de Entrenamiento:  0.8172041621066013
Score Catboost set de Validación:  0.8130045398648493
Score Catboost set de Prueba:  0.8137058902271526


In [44]:
start_time_cat = time.time()
cat_model_predict = cat_model.predict(feat_test_cat)
predict_time_cat = time.time() - start_time_cat
print(f"Predicting time CatBoost model: {predict_time_light:.4f} seconds")
print()
print(f"RMSE test CatBoost model: {rmse(target_test_cat, cat_model_predict)}")

Predicting time CatBoost model: 0.0990 seconds

RMSE test CatBoost model: 2026.4438200916509


#### Conclusiones modelo CatBoost

Al igual que el modelo LightGBM este algoritmo incluye dentro de sus parámetros métricas de evaluación de error y se puede indidcar la cantidad de iteraciones deseadas.
Usando métricas similares a LightGBM este modelo tiene una mejor evaluación de data sets y logra minimizar el error de la regresión con más eficiencia.
Incluso es más rápido en entrenarse que LightGBM,

### Modelo Random Forest Regressor
Para este modero de bosque de regresión usaremos los siguientes hiperparámetros:

n_estimators = este parámetro indica la cantidad de árboles que va a tener nuestro bosque.

max_depth = aqui establecemos que tan profundo va a ser cada árbol dentro de nuestro bosque.

max_leaf_nodes = aqui establecemos cuantos nodos finales vamos a tener por árbol.

random_state = establecemos que la data entre de manera aleatoria al momento de crear cada árbol de regresión.

In [38]:
forest_model = RandomForestRegressor(n_estimators=40, max_depth=15, max_leaf_nodes=60, random_state=12345)
start_time_forest = time.time()
forest_model.fit(feat_train, target_train)
training_time_forest = time.time() - start_time_forest
print(f"Training time RandomForestRegressor model: {training_time_forest:.4f} seconds")

Training time RandomForestRegressor model: 4.9284 seconds


In [39]:
print("Entrenamiento: ", forest_model.score(feat_train, target_train))
print("Validación: ", forest_model.score(feat_valid, target_valid))
print("Prueba: ", forest_model.score(feat_test, target_test))

Entrenamiento:  0.7437019624403625
Validación:  0.7403625239101862
Prueba:  0.7406437103699327


In [40]:
start_time_forest_predict = time.time()
forest_predict = forest_model.predict(feat_test)
time_predict_forest = time.time() - start_time_forest_predict
print(f"Predicting time RandomForestRegressor model: {time_predict_forest:.4f} seconds")
print()
print(f"RMSE test RandomForestRegressor model: {rmse(forest_predict, target_test)}")

Predicting time RandomForestRegressor model: 0.1230 seconds

RMSE test RandomForestRegressor model: 2391.021044473207


#### Conclusiones modelo RandomForestRegressor

Este modelo llega a iterar con la cantidad de árboles que le asignamos.
Aqui mientras más parámetros modifiquemos más robusto será el modelo pero más lento se vuelve.
Las valuaciones de los datasets dentro de este modelo no llegan a ser tan satisfactorios como los otros dos modelos.
También es el modelo más lento dentro de los 3 que estamos comparando.

## Conclusiones generales

Considerando los tiempos de entrenamiento y la métrica de error RMSE, el modelo que considero es el mejor es el CatBoost.
Este modelo tiene el tiempo más corto de entrenamiento y el error más pequeño.
Con la misma cantidad de recursos que LightGBM llega a minimizar el error de manera más eficiente.
Además de que podemos ahorrarnos el paso de la codificación One Hot para este tipo de modelo, a diferencia del modelo RandomForestRegressor.
Con un menor tiempo de entrenamiento y predicción, además de presentar la mejor eficiencia al iterar la mejor opción es CatBoost.